In [2]:
# https://krishansubudhi.github.io/deeplearning/2019/09/26/BertAttention.html

In [3]:
import torch
# from pytorch_transformers import BertConfig, BertTokenizer,  BertModel
import os
import json
import re
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from transformers import *
from tokenizers import *
from datasets import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import nltk
from nltk.data import load
import pickle

/home/hdh/PycharmProjects/huggingface_bert/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
model_path = "c09k_finetuned_bert2"
tokenizer_path = 'c09k_tokenizer_2'

In [33]:
?BertForSequenceClassification.from_pretrained

In [52]:
# load the model checkpoint
# config = BertConfig.from_pretrained(model_path, "checkpoint-6000")
# config.output_attentions = True
model = BertForSequenceClassification.from_pretrained(
    os.path.join(model_path, "checkpoint-6000"), return_dict=True, num_labels=18,
    output_attentions=True).to('cuda')
# load the tokenizer

loading configuration file c09k_finetuned_bert2/checkpoint-6000/config.json
Model config BertConfig {
  "_name_or_path": "c09k_finetuned_bert/checkpoint-3000",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LA

In [35]:
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_path, vocab_size=8000, local_files_only=True)
optimizer = AdamW(model.parameters(), lr=1e-5)
# optimizer = A AdamW(model1.parameters(), lr=1e-5)

Didn't find file c09k_tokenizer_2/added_tokens.json. We won't load it.
Didn't find file c09k_tokenizer_2/special_tokens_map.json. We won't load it.
Didn't find file c09k_tokenizer_2/tokenizer_config.json. We won't load it.
loading file c09k_tokenizer_2/vocab.txt
loading file c09k_tokenizer_2/tokenizer.json
loading file None
loading file None
loading file None
/home/hdh/PycharmProjects/huggingface_bert/venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
text_classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [19]:
train_data_df = pd.read_csv('data/train_C09K11_220715.txt', sep='\t')  # text와 라벨 파일
test_dataset_df = pd.read_csv('data/test_C09K11_220715.txt', sep='\t')
train_dataset = Dataset.from_pandas(train_data_df)  # Dataset 객체 생성
test_dataset = Dataset.from_pandas(test_dataset_df)
finetune_dataset = DatasetDict()  # DatasetDict 객체 생성
finetune_dataset['train'] = train_dataset
finetune_dataset['test'] = test_dataset
finetune_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9881
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5203
    })
})

In [22]:
# finetune_dataset['test']['text']

In [23]:
encoding = tokenizer(finetune_dataset['test']['text'][:16], return_tensors='pt', padding=True, truncation=True, max_length=512)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [26]:
# 토크나이저 함수 객체 생성
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512, padding=True)
def preprocess_function1(examples):
    return examples['label']

In [27]:
tokenized_finetune_dataset = finetune_dataset.map(preprocess_function, batched=True)  

100%|██████████| 6/6 [00:00<00:00,  7.03ba/s]


In [39]:
tokenized_finetune_dataset['test']['text'][:2]

['보안 인쇄물의 위변조 확인 방법', '보안 인쇄물의 위변조 확인 방법']

In [ ]:
model()

In [18]:
# training_args = TrainingArguments(output_dir="c09k_finetuned_bert")
training_args = TrainingArguments(
    output_dir="c09k_finetuned_bert_512",
    evaluation_strategy="epoch",    # 'steps': evaluate each `logging_steps`, 'epoch'  : each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_steps=500,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
sample_text = '보안 인쇄물의 위변조 확인 방법'
tok = tokenizer(sample_text, truncation=True, max_length=512, padding=True)
# tok
ids = torch.tensor(tokenizer.convert_tokens_to_ids(tok)).unsqueeze(0).to('cuda')
with torch.no_grad():
    output = model(ids)

In [71]:
# output['logits']
# output['attentions']
attentions = torch.cat(output['attentions']).to('cpu')
attentions.shape
attentions

tensor([[[[0.2548, 0.4683, 0.2769],
          [0.2732, 0.4830, 0.2438],
          [0.2737, 0.4782, 0.2480]],

         [[0.5247, 0.1906, 0.2847],
          [0.4911, 0.1865, 0.3224],
          [0.5113, 0.1720, 0.3167]],

         [[0.2249, 0.5777, 0.1974],
          [0.2619, 0.5478, 0.1903],
          [0.2197, 0.5911, 0.1892]],

         ...,

         [[0.3967, 0.3252, 0.2781],
          [0.3909, 0.3510, 0.2581],
          [0.3977, 0.3278, 0.2746]],

         [[0.1652, 0.3719, 0.4629],
          [0.1792, 0.3443, 0.4765],
          [0.1594, 0.3679, 0.4728]],

         [[0.3092, 0.5859, 0.1049],
          [0.3665, 0.5183, 0.1152],
          [0.3536, 0.5253, 0.1211]]],


        [[[0.4473, 0.2110, 0.3417],
          [0.4447, 0.2051, 0.3502],
          [0.4582, 0.2106, 0.3312]],

         [[0.4159, 0.3488, 0.2354],
          [0.4499, 0.3128, 0.2372],
          [0.4364, 0.3445, 0.2191]],

         [[0.3453, 0.2429, 0.4119],
          [0.3677, 0.2574, 0.3749],
          [0.4157, 0.2385, 0.34

In [46]:
tokenizer.convert_tokens_to_ids(tok)

[1, 1, 1]

### Plot Attention